In [4]:
transform_combinations = [
    # 1. Geometric + Intensity
    {   "zoom_and_crop": [1.05],
        "rotate_image": [-10],
        "axis_aligned_flip": ["horizontal"],
        
        "adjust_brightness": [5],
        "adjust_contrast": [1.1]
    },

    # 2. Geometric + Noise
    {   "zoom_and_crop": [0.95],
        "rotate_image": [15],
        "axis_aligned_flip": ["vertical"],
        
        "add_gaussian_noise": [0, 0.2],
        "add_speckle_noise": [0.1]
    },

    # 3. Occlusions + Intensity
    {
        "random_occlusions": [2, (30, 30)],
        "adjust_brightness": [-7],
        "apply_gamma_adjustment": [0.9],
        "adjust_contrast": [0.85]
    },

    # 4. Blur + Intensity
    {
        "gaussian_blur": [2],
        "adjust_brightness": [10],
        "apply_gamma_adjustment": [1.2],
        "adjust_contrast": [0.9]
    },

    # 5. Geometric + Advanced Adjustments
    {   "zoom_and_crop": [1.1],
        "rotate_image": [-20],
        "axis_aligned_flip": ["horizontal"],
        
        "bias_field_simulation": [0.2],
        "boundary_smoothing": [3]
    },

    # 6. Occlusions + Noise
    {
        "random_occlusions": [3, (20, 20)],
        "add_gaussian_noise": [0, 0.1],
        "add_speckle_noise": [0.05],
        "adjust_brightness": [-5]
    },

    # 7. Noise + Blur
    {
        "add_gaussian_noise": [0, 0.3],
        "add_speckle_noise": [0.2],
        "gaussian_blur": [3],
        "apply_gamma_adjustment": [1.1]
    },

    # 8. Advanced Adjustments
    {
        "bias_field_simulation": [0.3],
        "boundary_smoothing": [5],
        "histogram_equalization": [True],
        "adjust_brightness": [-8]
    },

    # 9. Geometric + Occlusions
    {   "zoom_and_crop": [1.05],
        "rotate_image": [10],
        "axis_aligned_flip": ["vertical"],
        "random_occlusions": [1, (40, 40)],
        
    },

    # 10. Mixed Transformations
    {   "zoom_and_crop": [0.9],
        "rotate_image": [-15],
        
        "add_gaussian_noise": [0, 0.2],
        "gaussian_blur": [1],
        "adjust_contrast": [1.2]
    }
]

In [93]:
parameter_ranges = {
    # Geometric Transformations
    "rotate_image": (-20, 20),  # Rotation: Range -20° to +20° (evenly sampled)
    "zoom_and_crop": (0.9, 1.1),  # Zoom: Range 0.9x to 1.1x (evenly sampled)
    # Random Occlusions under Geometric trans
    "random_occlusions": {  # Random Occlusions (only size matters here)
        "num_occlusions": (0.00001 , 6),  # number of occlusions:assign whole number
        "size": (20, 40),  # Size: Range 20x20 to 40x40 pixels (evenly sampled)
    },

    # Intensity Transformations
    "adjust_brightness": (-10, 10),  # Brightness: Range -10 to +10 (evenly sampled)
    "adjust_contrast": (0.8, 1.2),  # Contrast: Range 0.8 to 1.2 (evenly sampled)
    "apply_gamma_adjustment": (0.8, 1.2),  # Gamma Correction: Range 0.8 to 1.2 (evenly sampled)

    # Noise Injection
    "add_gaussian_noise": (0.1, 0.3),  # Gaussian Noise: std range 0.1 to 0.3 (evenly sampled)
    "add_speckle_noise": (0.05, 0.2),  # Speckle Noise: Variance range 0.05 to 0.2 (evenly sampled)

    # Blurring/Sharpening
    "gaussian_blur": (1, 3),  # Gaussian Blur: Sigma range 1 to 3 (evenly sampled)
    "gaussian_sharpening": (0.5, 1.5),  # Gaussian Sharpening: Amount range 0.5 to 1.5 (evenly sampled)

    

    # Advanced Adjustments
    "bias_field_simulation": (0.1, 0.3),  # Bias Field Simulation: Max bias 0.1 to 0.3 (evenly sampled)
    "boundary_smoothing": (0.0001, 5),  # Boundary Smoothing: Kernel size 1 to 5 (whole numbers)
}


In [94]:
def count_transformation_methods(transform_combinations):
    """
    Counts the occurrences of each transformation method in a set of transformation combinations.

    Args:
        transform_combinations (list): A list of dictionaries where each dictionary defines a transformation combination.

    Returns:
        list: A list of tuples where each tuple contains a transformation method and its total count.
    """
    from collections import Counter

    method_counter = Counter()

    for combination in transform_combinations:
        for method in combination.keys():
            method_counter[method] += 1

    # Convert to a list of tuples (method, count)
    method_counts = list(method_counter.items())

    return method_counts



method_counts = count_transformation_methods(transform_combinations)
print(method_counts)


[('zoom_and_crop', 5), ('rotate_image', 5), ('axis_aligned_flip', 4), ('adjust_brightness', 5), ('adjust_contrast', 4), ('add_gaussian_noise', 4), ('add_speckle_noise', 3), ('random_occlusions', 3), ('apply_gamma_adjustment', 3), ('gaussian_blur', 3), ('bias_field_simulation', 2), ('boundary_smoothing', 2), ('histogram_equalization', 1)]


In [95]:
def calculate_intervals(method_counts, total_samples, parameter_ranges, special_list=None):
    """
    Calculate interval sizes for each transformation method based on their ranges and total usage.

    Args:
        method_counts (list): A list of tuples where each tuple contains a transformation method and its count.
        total_samples (int): Total number of samples to process.
        parameter_ranges (dict): A dictionary of parameter ranges for each transformation method.

    Returns:
        dict: A dictionary containing interval sizes for each method.
    """
    
    if special_list is None:
        special_list = []
        
    overlap = set(parameter_ranges.keys()).intersection(set(special_list))
    if overlap:
        raise ValueError(f"The following methods appear in both parameter_ranges and special_list: {overlap}")

    # Ensure all methods in method_counts are covered in parameter_ranges or special_list
    all_methods = set([method for method, _ in method_counts])
    covered_methods = set(parameter_ranges.keys()).union(set(special_list))

    if not all_methods.issubset(covered_methods):
        missing_methods = all_methods - covered_methods
        raise ValueError(f"The following methods are missing from parameter_ranges or special_list: {missing_methods}")
    
    
    intervals = {}

    for method, count in method_counts:
        total_appearance_count = count * total_samples * 2  # Each method applied twice per sample

        if method in parameter_ranges:
            # Handle evenly sampled ranges
            if isinstance(parameter_ranges[method], tuple):
                range_start, range_end = parameter_ranges[method]
                interval_size = (range_end - range_start) / total_appearance_count
                intervals[method] = interval_size

            # Handle special cases (like random_occlusions)
            elif isinstance(parameter_ranges[method], dict):
                special_intervals = {}
                for sub_param, (start, end) in parameter_ranges[method].items():
                    interval_size = (end - start) / total_appearance_count
                    special_intervals[sub_param] = interval_size
                intervals[method] = special_intervals

    return intervals

total_samples = 300

# Example usage
special_list = [ "axis_aligned_flip", "histogram_equalization"]
intervals = calculate_intervals(method_counts, total_samples, parameter_ranges, special_list)
print(intervals)

{'zoom_and_crop': 6.666666666666668e-05, 'rotate_image': 0.013333333333333334, 'adjust_brightness': 0.006666666666666667, 'adjust_contrast': 0.00016666666666666663, 'add_gaussian_noise': 8.333333333333333e-05, 'add_speckle_noise': 8.333333333333334e-05, 'random_occlusions': {'num_occlusions': 0.003333327777777778, 'size': 0.011111111111111112}, 'apply_gamma_adjustment': 0.00022222222222222218, 'gaussian_blur': 0.0011111111111111111, 'bias_field_simulation': 0.00016666666666666666, 'boundary_smoothing': 0.004166583333333334}


In [14]:
import math
print(math.ceil(0.001))


1


In [103]:
import math
import random
def special_list_function(method, params, order, parameter_ranges, intervals, method_counts):
    """
    Handles special cases for transformation methods.
    Args:
        method (str): The method name.
        params (any): The parameters for the method.
        order (int): The current order number.
        parameter_ranges (dict): Parameter ranges for methods.
        intervals (dict): Calculated intervals for each transformation method.
        method_counts (dict): Count of occurrences for each method.
    Returns:
        any: Updated parameters for the method.
    """
    if method == "random_occlusions":
        # Special handling for random_occlusions using intervals and lower parameter ranges
        num_occlusions = float(parameter_ranges[method]["num_occlusions"][0]) + float(((order - 1) * intervals[method]["num_occlusions"] / method_counts[method]))
        size_variation = random.uniform(1, 5)  # Add random variation factor between 1 and 5
        size_variation_sec = random.uniform(1, 3)  # Add random variation factor between 1 and 5
        size = (
            parameter_ranges[method]["size"][0] + ((order - 1) * intervals[method]["size"] / method_counts[method]) * size_variation,
            parameter_ranges[method]["size"][0] + ((order - 1) * intervals[method]["size"] / method_counts[method]) * size_variation_sec
        )
        return [math.ceil(num_occlusions), size]
    elif method == "axis_aligned_flip":
        # Alternate between "vertical" and "horizontal" based on the order
        return ["vertical"] if order % 2 == 1 else ["horizontal"]
    elif method == "histogram_equalization":
        # Always return True for histogram_equalization
        return True
    return params

def process_transformations(order, transform_combinations, intervals, special_list, parameter_ranges, method_counts):
    """
    Process a single image-mask pair with specified transformation combinations and intervals.

    Args:
        order (int): The order number of the transformation (e.g., 1 to total_samples*2).
        transform_combinations (list): List of transformation combination dictionaries.
        intervals (dict): Calculated intervals for each transformation method.
        special_list (list): List of special methods that require custom handling.
        parameter_ranges (dict): Parameter ranges for methods.
        method_counts (dict): Count of occurrences for each method.

    Returns:
        list: Modified transform_combinations with updated parameters.
    """
    # Ensure method_counts is always a dictionary
    if isinstance(method_counts, list):
        method_counts = dict(method_counts)
        
        
    updated_combinations = []
    current_method_orders = {method: 0 for method in method_counts}
    
    

    for combination in transform_combinations:
        updated_combination = {}
        for method, params in combination.items():
            current_order = (order - 1) * method_counts[method] + current_method_orders[method] + 1
            if method in special_list:
                # Call special_list_function for special handling
                updated_combination[method] = special_list_function(method, params, current_order, parameter_ranges, intervals, method_counts)
            elif method in intervals:
                # Regular calculation process with base as lower parameter range
                if isinstance(intervals[method], dict):
                    updated_combination[method] = {}
                    for sub_param in intervals[method]:
                        try:
                            # Check if params[sub_param] exists and is a list
                            if sub_param not in params or not isinstance(params[sub_param], list):
                                raise ValueError(f"Parameter '{sub_param}' for method '{method}' must be a list.")
                            updated_combination[method][sub_param] = parameter_ranges[method][sub_param][0] + ((current_order - 1) * intervals[method][sub_param] / method_counts[method])
                        except Exception as e:
                            raise ValueError(f"Error with method '{method}' and parameter '{sub_param}': {e}")
                else:
                    updated_combination[method] = parameter_ranges[method][0] + ((current_order - 1) * intervals[method] / method_counts[method])
            else:
                raise ValueError(f"Method {method} is not recognized in intervals or special_list.")

            current_method_orders[method] = current_method_orders[method] + 1

        updated_combinations.append(updated_combination)

    return updated_combinations

special_list.append("random_occlusions")
updated_combinations = process_transformations(
    1, transform_combinations, intervals, special_list, parameter_ranges, method_counts
)

In [100]:
random.uniform(1, 5)

2.5885965549332948

In [104]:
def print_updated_combinations(updated_combinations):
    """
    Helper function to print updated combinations in a readable format.

    Args:
        updated_combinations (list): List of updated transformation combinations.
    """
    print("Updated Combinations:")
    for idx, combination in enumerate(updated_combinations, start=1):
        print(f"Combination {idx}:")
        for method, params in combination.items():
            print(f"  {method}: {params}")
    print()
    
print_updated_combinations(updated_combinations)

Updated Combinations:
Combination 1:
  zoom_and_crop: 0.9
  rotate_image: -20.0
  axis_aligned_flip: ['vertical']
  adjust_brightness: -10.0
  adjust_contrast: 0.8
Combination 2:
  zoom_and_crop: 0.9000133333333333
  rotate_image: -19.997333333333334
  axis_aligned_flip: ['horizontal']
  add_gaussian_noise: 0.1
  add_speckle_noise: 0.05
Combination 3:
  random_occlusions: [1, (20.0, 20.0)]
  adjust_brightness: -9.998666666666667
  apply_gamma_adjustment: 0.8
  adjust_contrast: 0.8000416666666668
Combination 4:
  gaussian_blur: 1.0
  adjust_brightness: -9.997333333333334
  apply_gamma_adjustment: 0.8000740740740742
  adjust_contrast: 0.8000833333333334
Combination 5:
  zoom_and_crop: 0.9000266666666666
  rotate_image: -19.994666666666667
  axis_aligned_flip: ['vertical']
  bias_field_simulation: 0.1
  boundary_smoothing: 0.0001
Combination 6:
  random_occlusions: [1, (20.006762693994325, 20.010872881024625)]
  add_gaussian_noise: 0.10002083333333334
  add_speckle_noise: 0.05002777777777

In [2]:
import os

def explore_folder_mismatch(img_folder, mask_folder):
    """
    Compares the contents of two folders and identifies mismatched files.

    Args:
        img_folder (str): Path to the folder containing images.
        mask_folder (str): Path to the folder containing masks.

    Returns:
        None
    """
    # List files in both folders
    img_files = sorted(os.listdir(img_folder))
    mask_files = sorted(os.listdir(mask_folder))

    # Convert file lists to sets
    img_set = set(img_files)
    mask_set = set(mask_files)

    # Identify mismatches
    only_in_img = img_set - mask_set
    only_in_mask = mask_set - img_set
    common_files = img_set & mask_set

    # Print mismatched files
    print("Files only in image folder:")
    for file in only_in_img:
        print(f"  {file}")
    print("\nFiles only in mask folder:")
    for file in only_in_mask:
        print(f"  {file}")
    print("\nCommon files:")
    for file in common_files:
        print(f"  {file}")

    # Check filename patterns (e.g., nnU-Net naming convention)
    print("\nChecking filename patterns:")
    img_pattern_mismatches = [file for file in img_files if "_0000" not in file]
    if img_pattern_mismatches:
        print("  Files in image folder without '_0000':")
        for file in img_pattern_mismatches:
            print(f"    {file}")

    mask_pattern_mismatches = [file for file in mask_files if "_0000" not in file]
    if mask_pattern_mismatches:
        print("  Files in mask folder without '_0000':")
        for file in mask_pattern_mismatches:
            print(f"    {file}")

    # Summary
    print("\nSummary:")
    print(f"Total files in image folder: {len(img_files)}")
    print(f"Total files in mask folder: {len(mask_files)}")
    print(f"Files only in image folder: {len(only_in_img)}")
    print(f"Files only in mask folder: {len(only_in_mask)}")
    print(f"Common files: {len(common_files)}")

# Example Usage
img_folder = "/data/bruce/unet/Dataset002_AnychestTry/imagesTr"
mask_folder = "/data/bruce/unet/Dataset002_AnychestTry/labelsTr"
explore_folder_mismatch(img_folder, mask_folder)


Files only in image folder:
  t_5074_a_1_0000.nii.gz
  t_15131_a_1_0000.nii.gz
  t_12717_a_1_0000.nii.gz
  t_11026_a_1_0000.nii.gz
  t_17959_a_1_0000.nii.gz
  t_18730_a_1_0000.nii.gz
  t_8222_a_1_0000.nii.gz
  t_19245_a_1_0000.nii.gz
  t_8458_a_1_0000.nii.gz
  t_15629_a_1_0000.nii.gz
  t_11303_a_1_0000.nii.gz
  t_6132_a_1_0000.nii.gz
  t_16384_a_1_0000.nii.gz
  t_9941_a_1_0000.nii.gz
  t_11066_a_1_0000.nii.gz
  t_9386_a_1_0000.nii.gz
  t_8837_a_1_0000.nii.gz
  t_8565_a_1_0000.nii.gz
  t_12387_a_1_0000.nii.gz
  t_916_a_1_0000.nii.gz
  t_1680_a_1_0000.nii.gz
  t_18477_a_1_0000.nii.gz
  t_13888_a_1_0000.nii.gz
  t_3710_a_1_0000.nii.gz
  t_7220_a_1_0000.nii.gz
  t_1270_a_1_0000.nii.gz
  t_2620_a_1_0000.nii.gz
  t_11476_a_1_0000.nii.gz
  t_3138_a_1_0000.nii.gz
  t_14376_a_2_0000.nii.gz
  t_2275_a_1_0000.nii.gz
  t_184_a_1_0000.nii.gz
  t_6960_a_1_0000.nii.gz
  t_11432_a_1_0000.nii.gz
  t_17202_a_1_0000.nii.gz
  t_12040_a_1_0000.nii.gz
  t_12520_a_1_0000.nii.gz
  t_13720_a_1_0000.nii.gz
  t_

In [3]:
def explore_folder_mismatch(img_folder, mask_folder):
    """
    Compares the contents of two folders and identifies mismatched files after removing `_0000` from image filenames.

    Args:
        img_folder (str): Path to the folder containing images.
        mask_folder (str): Path to the folder containing masks.

    Returns:
        None
    """
    # List files in both folders
    img_files = sorted(os.listdir(img_folder))
    mask_files = sorted(os.listdir(mask_folder))

    # Remove '_0000' suffix from image filenames
    img_files_modified = [file.replace("_0000", "") for file in img_files]

    # Convert file lists to sets
    img_set = set(img_files_modified)
    mask_set = set(mask_files)

    # Identify mismatches
    only_in_img = img_set - mask_set
    only_in_mask = mask_set - img_set
    common_files = img_set & mask_set

    # Print mismatched files
    print("Files only in image folder (after removing '_0000'):")
    for file in only_in_img:
        print(f"  {file}")
    print("\nFiles only in mask folder:")
    for file in only_in_mask:
        print(f"  {file}")
    print("\nCommon files:")
    for file in common_files:
        print(f"  {file}")

    # Summary
    print("\nSummary:")
    print(f"Total files in image folder (modified): {len(img_files_modified)}")
    print(f"Total files in mask folder: {len(mask_files)}")
    print(f"Files only in image folder: {len(only_in_img)}")
    print(f"Files only in mask folder: {len(only_in_mask)}")
    print(f"Common files: {len(common_files)}")


# Example Usage
img_folder = "/data/bruce/unet/Dataset002_AnychestTry/imagesTr"
mask_folder = "/data/bruce/unet/Dataset002_AnychestTry/labelsTr"
explore_folder_mismatch(img_folder, mask_folder)


Files only in image folder (after removing '_0000'):

Files only in mask folder:

Common files:
  t_7293_a_1.nii.gz
  t_12952_a_1.nii.gz
  t_4603_a_1.nii.gz
  t_5190_a_1.nii.gz
  t_13720_a_1.nii.gz
  t_15906_a_1.nii.gz
  t_16666_a_1.nii.gz
  t_10233_a_1.nii.gz
  t_14728_a_1.nii.gz
  t_9015_a_1.nii.gz
  t_9714_a_1.nii.gz
  t_14369_a_1.nii.gz
  t_17367_a_1.nii.gz
  t_18196_a_1.nii.gz
  t_17197_a_1.nii.gz
  t_15010_a_1.nii.gz
  t_18730_a_1.nii.gz
  t_18055_a_1.nii.gz
  t_2296_a_1.nii.gz
  t_2478_a_1.nii.gz
  t_6440_a_1.nii.gz
  t_13244_a_1.nii.gz
  t_1385_a_1.nii.gz
  t_13711_a_1.nii.gz
  t_3799_a_1.nii.gz
  t_4023_c_1.nii.gz
  t_5359_a_1.nii.gz
  t_17237_a_1.nii.gz
  t_11303_a_1.nii.gz
  t_6138_a_1.nii.gz
  t_8785_a_1.nii.gz
  t_10152_a_1.nii.gz
  t_9820_a_1.nii.gz
  t_7262_a_1.nii.gz
  t_3686_a_1.nii.gz
  t_16354_a_1.nii.gz
  t_15965_a_1.nii.gz
  t_8973_a_1.nii.gz
  t_5816_a_1.nii.gz
  t_5885_a_1.nii.gz
  t_4514_b_1.nii.gz
  t_184_a_1.nii.gz
  t_7673_a_1.nii.gz
  t_16107_a_1.nii.gz
  t_

In [2]:
import os
import shutil

def copy_dataset(source_paths, dest_path):
    """
    Copies files and folders from multiple source paths to a destination path.

    Args:
        source_paths (list): List of source folder paths to copy from.
        dest_path (str): Destination folder path.

    Returns:
        None
    """
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    for source in source_paths:
        if not os.path.exists(source):
            print(f"Source path does not exist: {source}")
            continue

        for item in os.listdir(source):
            source_item = os.path.join(source, item)
            dest_item = os.path.join(dest_path, item)

            if os.path.isdir(source_item):
                shutil.copytree(source_item, dest_item, dirs_exist_ok=True)
            else:
                shutil.copy2(source_item, dest_item)

# Define source and destination paths
source_imagesTr = [
    "/data/bruce/unet/Dataset002_AnychestTryAug/imagesTr",
    "/data/bruce/unet/Dataset002_AnychestTry/imagesTr",
]
source_labelsTr = [
    "/data/bruce/unet/Dataset002_AnychestTryAug/labelsTr",
    "/data/bruce/unet/Dataset002_AnychestTry/labelsTr",
]
source_imagesTs = "/data/bruce/unet/Dataset002_AnychestTry/imagesTs"

dest_imagesTr = "/data/bruce/unet/Dataset003_AnychestTry/imagesTr"
dest_labelsTr = "/data/bruce/unet/Dataset003_AnychestTry/labelsTr"
dest_imagesTs = "/data/bruce/unet/Dataset003_AnychestTry/imagesTs"

# Copy datasets
print("Copying imagesTr...")
copy_dataset(source_imagesTr, dest_imagesTr)

print("Copying labelsTr...")
copy_dataset(source_labelsTr, dest_labelsTr)

print("Copying imagesTs...")
copy_dataset([source_imagesTs], dest_imagesTs)

print("Dataset copying completed!")

Copying imagesTr...
Copying labelsTr...
Copying imagesTs...


Dataset copying completed!


In [ ]:
import os
import nibabel as nib
import numpy as np
from skimage.exposure import match_histograms

def histogram_match_nifti(source_path, reference_path, output_path):
    # Load NIfTI files
    source_img = nib.load(source_path)
    reference_img = nib.load(reference_path)
    
    source_data = source_img.get_fdata()
    reference_data = reference_img.get_fdata()
    
   
   
   
   
    
    # Save the matched image
    matched_img = nib.Nifti1Image(matched_data, affine=source_img.affine)
    nib.save(matched_img, output_path)

# Example usage
source_path = "/data/bruce/test/CXR/JPCLN001_0000.nii.gz"  # Path to the source NIfTI file
reference_path = "/data/bruce/test/DRR/t_40_a_1_0000.nii.gz"  # Path to the reference NIfTI file
output_folder = "/data/bruce/test/Histo_trans"  # Path to save the output NIfTI file
output_name = "JPCLN001_0000_hisnew.nii.gz"

os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_name)

histogram_match_nifti(source_path, reference_path, output_path)
